# **Reducing Traffic Mortality in the USA**

Note that the data used for this report is from the year 2022

By **Mike Musas**    |    *August 10, 2024*

## **1. Introduction**

Traffic mortality has been a public health issue in the United States, with thousands of individuals dying each year due to various factors contributing to road accidents. This project aims to identify the leading causes of traffic fatalities and recommend effective strategies to reduce the number of deaths on American roads.

The project will examine the leading contributors to traffic fatalities, including speeding, drunk driving, and distracted driving. Speeding remains one of the most significant factors, as higher speeds reduce the driver’s ability to react to sudden changes and increase the severity of accidents. Drunk driving is another major cause, with impaired judgment and delayed reaction times leading to a higher likelihood of crashes. Distracted driving, often due to mobile phone use or other in-vehicle distractions, has also emerged as a critical concern in recent years.

**The project also aims to answer the following questions:**

* How has the total number of traffic fatalities changed over the past years?
* What are the age and gender distributions of traffic mortality victims?
* How does traffic mortality vary by state or region?
* Do the number of fatalities correlates with the population of regions?
* How does the fatality rate varies between rural and urban areas?
* What are the leading causes of traffic fatalities in the USA in 2022?

To answer all these questions, we are making use of Tableau desktop (for data visualization and also to create our report) and Bigquery (to create our database and store all the tables we are using).

## **2. About the Data**

### **2.a Database and Source**

The data used to answer the project’s questions come from the [National Highway Traffic Safety Administration](https://www.nhtsa.gov/file-downloads?p=nhtsa/downloads/FARS/) or NHTSA website. The data is comprised of 33 tables. However, 8 tables were used to answer our questions. Before doing our analysis, the data had to go through a cleaning process where the necessary number of columns were selected from every table.

### **2.b Entity-Relationship Diagram (ERD)**
 
The image below shows the entity-relationship diagram or ERD of the database used in this project. The ERD picture the relationship below all the tables in the database. This helps us understand the structure of the database and also understand how we can link different tables together. Note that the ERD below only shows a couple of columns for some of the tables. The reason behind this is to make sure everything is visible on the image.


<img src="Assets/msba_capston_diagram.png" alt="drawing" width="1000"/>

<p>.</p>

## Exploratory Data Analysis

### 1. Quick Numbers

#### 1.a Lives lost on U.S. roads in 2022

In [32]:
# Lives lost on U.S. roads in 2022
total_fatalities = client.query(
    """
    SELECT
        SUM(fatals) AS total_fatalities
    FROM NHTSA.accidents
    """
)

In [33]:
total_fatalities.to_dataframe()

,total_fatalities
0,42188


#### 1.b Total number of accidents on U.S. roads in 2022

In [55]:
# Total number of accidents on U.S. roads in 2022
total_accidents = client.query(
    """
    SELECT
        COUNT(*) AS total_accidents
    FROM NHTSA.accidents
    """
)

In [56]:
total_accidents.to_dataframe()

,total_accidents
0,38918


#### 1.c Average Age

In [26]:
average_age = client.query(
    """
    SELECT
        ROUND(AVG(p.age)) AS average_age
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.persons AS p
        ON a.st_case = p.st_case
    WHERE p.age >= 0 AND p.age <= 120;
    """
)

In [34]:
average_age.to_dataframe()

,average_age
0,40.0


### 2. Demographic Analysis

#### 2.a What is the gender distribution of traffic mortality victims?

##### Gender

In [38]:
# Gender distribution of traffic mortality victims
fatalities_by_gender = client.query(
    """
    SELECT
        p.sex,
        COUNT(*) AS total_fatalities,
        ROUND(COUNT(*) / (
            SELECT
                COUNT(*) AS total_fatalities
            FROM NHTSA.accidents AS a
            INNER JOIN NHTSA.persons AS p
                ON a.st_case = p.st_case
            WHERE p.doa IN ('Died at Scene', 'Died En Route')
            AND p.sex IN ('Male', 'Female')
            AND p.age < 120
        ), 4) AS percentage_fatalities
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.persons AS p
        ON a.st_case = p.st_case
    WHERE p.doa IN ('Died at Scene', 'Died En Route')
            AND p.sex IN ('Male', 'Female')
            AND p.age < 120
    GROUP BY p.sex;
    """
)

In [39]:
fatalities_by_gender.to_dataframe()

,sex,total_fatalities,percentage_fatalities
0,Male,17725,0.732
1,Female,6489,0.268


#### 2.b What is the age distribution of traffic mortality victims?

##### Age

In [44]:
# Age distribution of traffic mortality victims
fatalities_by_age = client.query(
    """
    SELECT
        CASE
            WHEN p.age >= 0 AND p.age < 12 THEN '0 >= Age < 12'
            WHEN p.age >= 12 AND p.age < 24 THEN '12 >= Age < 24'
            WHEN p.age >= 24 AND p.age < 36 THEN '24 >= Age < 36'
            WHEN p.age >= 36 AND p.age < 48 THEN '36 >= Age < 48'
            WHEN p.age >= 48 AND p.age < 60 THEN '48 >= Age < 60'
            WHEN p.age >= 60 AND p.age < 72 THEN '60 >= Age < 72'
            WHEN p.age >= 72 AND p.age < 84 THEN '72 >= Age < 84'
            WHEN p.age >= 84 AND p.age < 96 THEN '84 >= Age < 96'
            WHEN p.age >= 96 AND p.age < 108 THEN '98 >= Age < 108'
        END AS age_range,
        COUNT(*) AS total_fatalities
    
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.persons AS p
        ON a.st_case = p.st_case
    WHERE
        p.doa IN ('Died at Scene', 'Died En Route')
        AND p.age < 120
    GROUP BY age_range
    ORDER BY total_fatalities DESC;
    """
)

In [45]:
fatalities_by_age.to_dataframe()

,age_range,total_fatalities
0,24 >= Age < 36,6206
1,36 >= Age < 48,4794
2,48 >= Age < 60,4074
3,12 >= Age < 24,3961
4,60 >= Age < 72,3125
5,72 >= Age < 84,1397
6,84 >= Age < 96,346
7,0 >= Age < 12,322
8,98 >= Age < 108,16


#### 2.c What are the socio-economic backgrounds of the victims?

In [41]:
fatalities_by_race = client.query(
    """
    SELECT
        r.race,
        COUNT(*) AS total_fatalities
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.persons AS p
        ON a.st_case = p.st_case
    INNER JOIN NHTSA.race AS r
        ON p.st_case = r.st_case
        AND p.veh_no = r.veh_no
        AND p.per_no = r.per_no
    WHERE
        p.doa IN ('Died at Scene', 'Died En Route')
    GROUP BY r.race;
    """
)

In [42]:
fatalities_by_race.to_dataframe()

,race,total_fatalities
0,Black or African American,3857
1,White,16131
2,Unknown,2789
3,Other Indian (includes South and Central Ameri...,52
4,Asian Indian,68
5,Multiple Races Unspecified,61
6,Korean,14
7,"Asian or Pacific Islander, no specific (indivi...",79
8,Other Race,465
9,North American Indian or Alaska Native,464


#### 2.d Is there a correlation between traffic mortality rates and urban versus rural settings?

In [46]:
urban_vs_rural_fatalities = client.query(
    """
    SELECT
        rur_urb,
        COUNT(*) AS total_accidents,
        ROUND(COUNT(*) / (SELECT COUNT(*) FROM NHTSA.accidents WHERE rur_urb != 'Unknown'), 2) AS percentage_accidents,
        SUM(fatals) AS total_fatalities,
        ROUND(SUM(fatals) / (SELECT SUM(fatals) FROM NHTSA.accidents WHERE rur_urb != 'Unknown'), 4) AS percentage_fatalities
    FROM NHTSA.accidents
    WHERE rur_urb != 'Unknown'
    GROUP BY rur_urb;
    """
)

In [47]:
urban_vs_rural_fatalities.to_dataframe()

,rur_urb,total_accidents,percentage_accidents,total_fatalities,percentage_fatalities
0,Urban,23366,0.6,24898,0.5928
1,Rural,15378,0.4,17106,0.4072


### 3. Causes and Patterns

#### 3.a What are the primary causes of traffic fatalities (e.g., speeding, drunk driving, distracted driving, etc.)?

##### Distractions

In [35]:
fatalities_by_distractions = client.query(
    """
    SELECT
        d.drdistract,
        COUNT(*) AS total_accidents,
        SUM(fatals) AS total_fatalities
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.vehicles AS v
        ON a.st_case = v.st_case
    INNER JOIN NHTSA.distractions AS d
        ON v.st_case = d.st_case
        AND v.veh_no = d.veh_no
    WHERE d.drdistract != 'Unknown'
    GROUP BY d.drdistract
    ORDER BY total_fatalities DESC;
    """
)

In [36]:
fatalities_by_distractions.to_dataframe()

,drdistract,total_accidents,total_fatalities
0,Not Distracted,13662,15011
1,"Inattention (Inattentive), Details Unknown",1024,1123
2,"Distraction (Distracted), Details Unknown",349,379
3,No Driver Present/Unknown if Driver present,304,338
4,"Distracted by Outside Person, Object or Event",226,243
5,Distraction/Inattention,147,152
6,While Manipulating Mobile Phone,122,130
7,Other Mobile Phone Related,98,107
8,While Using Other Component/Controls Integral ...,102,106
9,By Other Occupant(s),99,105


#### 3.b How do weather conditions affect traffic mortality rates?

In [25]:
fatalities_by_weather_conditions = client.query(
    """
    SELECT
        w.weather,
        COUNT(*) as total_accidents,
        SUM(a.fatals) AS fatalities
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.weather AS w
        ON a.st_case = w.st_case
    WHERE w.weather != 'Unknown'
    GROUP BY w.weather;
    """
)

In [26]:
fatalities_by_weather_conditions.to_dataframe()

,weather,total_accidents,fatalities
0,Cloudy,5085,5573
1,Clear,29282,31731
2,Rain,2419,2599
3,"Fog, Smog, Smoke",416,451
4,Severe Crosswinds,92,106
5,Reported as Unknown,191,203
6,Freezing Rain or Drizzle,54,55
7,"Blowing Sand, Soil, Dirt",10,15
8,Snow,370,400
9,Blowing Snow,59,70


#### 3.c What road conditions or types (e.g., highways, local roads) are most associated with traffic fatalities?

##### Exceeded the speed limit

In [54]:
accidents_by_speeding = client.query(
    """
    SELECT
        CASE
            WHEN v.speedrel IN ('Yes, Exceeded Speed Limit', 'Yes, Racing', 'Yes, Specifics Unknown', 'Yes, Too Fast for Conditions') THEN 'Was Speeding'
            WHEN v.speedrel = 'No' THEN 'Was Not Speeding'
            ELSE 'Unknown'
        END AS speeding,
        COUNT(*) AS total_accidents,
        SUM(fatals) AS total_fatalities,
        ROUND(COUNT(*) / (
            SELECT
                COUNT(*)
            FROM NHTSA.accidents
            INNER JOIN NHTSA.vehicles
                USING(st_case)
            WHERE speedrel NOT IN ('Reported as Unknown', 'No Driver Present/Unknown if Driver Present')
            ), 4) AS percentage_accidents
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.vehicles AS v
        ON a.st_case = v.st_case
    WHERE v.speedrel NOT IN ('Reported as Unknown', 'No Driver Present/Unknown if Driver Present', 'Unknown')
    GROUP BY speeding
    """
)

In [55]:
accidents_by_speeding.to_dataframe()

,speeding,total_accidents,total_fatalities,percentage_accidents
0,Was Not Speeding,34946,38577,0.785
1,Was Speeding,9574,10714,0.215


#### 3.d Violations

##### Violations

Look at the percentage of accidents caused by violations compared to those accidents which were not caused by any violation

In [12]:
fatalities_by_violations = client.query(
    """
    SELECT
        vl.violation,
        COUNT(*) AS total_accidents,
        SUM(a.fatals) AS total_fatalities
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.vehicles AS v
        ON a.st_case = v.st_case
    INNER JOIN NHTSA.violations AS vl
        ON v.st_case = vl.st_case
        AND v.veh_no = vl.veh_no
    WHERE vl.violation NOT IN ('Unknown', 'Unknown Violation(s)')
    GROUP BY vl.violation
    ORDER BY total_accidents DESC;
    """
)

In [13]:
fatalities_by_violations.to_dataframe()

,violation,total_accidents,total_fatalities
0,Manslaughter or homicide,1106,1251
1,Driving while intoxicated (alcohol or drugs) o...,1030,1152
2,Fail to yield generally,487,506
3,"Inattentive, careless, improper driving, drivi...",450,478
4,Other driver license violations,432,468
...,...,...,...
64,Motorcycle lane violations (more than two per ...,1,1
65,Parking,1,1
66,"Fail to obey police, fireman, authorized perso...",1,1
67,"Energy speed (exceeding 55 mph, non-pointable)",1,1


#### 3.e Driver Impairments

In [35]:
# Total fatalities caused by driver impairments
total_fatalities_by_driver_impairment = client.query(
    """
    SELECT
        d.drimpair,
        SUM(a.fatals) total_fatalities_by_impairments
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.vehicles AS v
        ON a.st_case = v.st_case
    INNER JOIN NHTSA.driver_impairment AS d
        ON v.st_case = d.st_case
        AND v.veh_no = d.veh_no
    WHERE d.drimpair NOT IN ('Not Reported', 'Reported as Unknown if Impaired', 'No Driver Present/Unknown if Driver Present')
    GROUP BY d.drimpair
    ORDER BY total_fatalities_by_impairments DESC;
    """
)

In [36]:
total_fatalities_by_driver_impairment.to_dataframe()

,drimpair,total_fatalities_by_impairments
0,None/Apparently Normal,19122
1,"Under the Influence of Alcohol, Drugs or Medic...",6370
2,"Ill, Blackout",544
3,Asleep or Fatigued,526
4,Physical Impairment - No Details,502
5,Other Physical Impairment,283
6,"Emotional (depressed, angry, disturbed, etc.)",196
7,Deaf/Hard of Hearing,12
8,Blind/Low Vision,5
9,Paraplegic or in a Wheelchair,3


In [8]:
# Total accidents caused by driver impairments
total_accidents_by_driver_impairment = client.query(
    """
    SELECT
        d.drimpair,
        COUNT(*) total_accidents_by_impairments
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.vehicles AS v
        ON a.st_case = v.st_case
    INNER JOIN NHTSA.driver_impairment AS d
        ON v.st_case = d.st_case
        AND v.veh_no = d.veh_no
    WHERE d.drimpair NOT IN ('Not Reported', 'Reported as Unknown if Impaired', 'No Driver Present/Unknown if Driver Present')
    GROUP BY d.drimpair
    ORDER BY total_accidents_by_impairments DESC;
    """
)

In [9]:
total_accidents_by_driver_impairment.to_dataframe()

,drimpair,total_accidents_by_impairments
0,None/Apparently Normal,17478
1,"Under the Influence of Alcohol, Drugs or Medic...",5650
2,"Ill, Blackout",524
3,Asleep or Fatigued,484
4,Physical Impairment - No Details,460
5,Other Physical Impairment,244
6,"Emotional (depressed, angry, disturbed, etc.)",187
7,Deaf/Hard of Hearing,11
8,Blind/Low Vision,5
9,Paraplegic or in a Wheelchair,3


#### 3.f Drugs

In [48]:
# Total accidents by drugs involvement
total_accidents_by_drugs = client.query(
    """
    SELECT
        d.drugres,
        COUNT(d.drugres) AS total_drugs_involved
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.persons AS p
        ON a.st_case = p.st_case
    INNER JOIN NHTSA.drugs AS d
        ON p.st_case = d.st_case
        AND p.veh_no = d.veh_no
        AND p.per_no = d.per_no
    WHERE d.drugres NOT IN ('Not Reported', 'Reported as Unknown if Impaired', 'Unknown')
    GROUP BY d.drugres
    ORDER BY total_drugs_involved DESC;
    """
)

In [49]:
total_accidents_by_drugs.to_dataframe()

,drugres,total_drugs_involved
0,Test Not Given,50469
1,"Tested, No Drugs Found/Negative",12955
2,Delta 9-tetrahydrocannabinol [THC],4184
3,11-nor-9-carboxy-delta-9- tetrahydrocannabinol...,3603
4,Amphetamine,2895
...,...,...
256,(Synthetic Cannabinoid) PB-22 (QUPIC),1
257,Barbital,1
258,Loratadine,1
259,Furethidine,1


### Roads

##### Traffic way description

In [45]:
fatalities_by_traffic_way = client.query(
    """
    SELECT
        v.vtrafway AS traffice_way_description,
        COUNT(*) AS total_accidents,
        SUM(a.fatals) AS total_fatalities
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.vehicles AS v
        ON a.st_case = v.st_case
    WHERE v.vtrafway NOT IN ('Unknown', 'Reported as Unknown')
    GROUP BY v.vtrafway;
    """
)

In [46]:
fatalities_by_traffic_way.to_dataframe()

,traffice_way_description,total_accidents,total_fatalities
0,"Two-Way, Divided, Unprotected Median",9458,10471
1,"Two-Way, Not Divided",25402,28087
2,One-Way Trafficway,620,670
3,"Two-Way, Not Divided With a Continuous Left-Tu...",3433,3713
4,Non-Trafficway or Driveway Access,447,478
5,"Two-Way, Divided, Positive Median Barrier",6712,7647
6,Entrance/Exit Ramp,873,933
7,"Two-Way Divided, Unknown if Unprotected Median...",235,272


##### Roadway Alignments

In [47]:
fatalities_by_roadway_alignment = client.query(
    """
    SELECT
        CASE
            WHEN v.valign IN ('Curve Left', 'Curve Right', 'Curve - Unknown Direction') THEN 'Curve'
            ELSE v.valign
        END AS roadway_alignment,
        COUNT(*) AS total_accidents,
        SUM(a.fatals) AS total_fatalities
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.vehicles AS v
        ON a.st_case = v.st_case
    WHERE v.valign NOT IN ('Unknown', 'Reported as Unknown')
    GROUP BY roadway_alignment;
    """
)

In [48]:
fatalities_by_roadway_alignment.to_dataframe()

,roadway_alignment,total_accidents,total_fatalities
0,Straight,36506,40446
1,Curve,9568,10624
2,Non-Trafficway or Driveway Access,447,478


##### Roadway Surface Condition

In [49]:
fatalities_by_roadway_surface_condition = client.query(
    """
    SELECT
        v.vsurcond,
        COUNT(*) AS total_accidents,
        SUM(a.fatals) AS total_fatalities
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.vehicles AS v
        ON a.st_case = v.st_case
    WHERE v.vsurcond NOT IN ('Unknown', 'Reported as Unknown')
    GROUP BY v.vsurcond;
    """
)

In [50]:
fatalities_by_roadway_surface_condition.to_dataframe()

,vsurcond,total_accidents,total_fatalities
0,Dry,40666,45051
1,Non-Trafficway or Driveway Access,447,478
2,Wet,4652,5113
3,Ice/Frost,390,421
4,"Mud, Dirt or Gravel",100,103
5,Sand,10,10
6,Snow,305,420
7,Water (Standing or Moving),61,70
8,Slush,77,84
9,Other,41,44


### 5. Time & Dates

#### At what times of day and days of the week do most traffic fatalities occur?

In [56]:
# Day of the week and Time distributions when most of the traffic fatalities occurred
fatalities_by_hours = client.query(
    """
    WITH fatalities_hours_days AS (
        SELECT
            day_week,
            day_numbers,
            hour,
            COUNT(*) AS total_accidents,
            SUM(fatals) AS total_fatalities
        FROM (
            SELECT
                day_week,
                hour,
                CASE
                    WHEN day_week = 'Monday' THEN 1
                    WHEN day_week = 'Tuesday' THEN 2
                    WHEN day_week = 'Wednesday' THEN 3
                    WHEN day_week = 'Thursday' THEN 4
                    WHEN day_week = 'Friday' THEN 5
                    WHEN day_week = 'Saturday' THEN 6
                    WHEN day_week = 'Sunday' THEN 7
                END AS day_numbers,
                fatals
            FROM NHTSA.accidents
             ) AS accidents
        GROUP BY day_week, day_numbers, hour
        ORDER BY day_numbers, hour
    )
    SELECT
        day_week,
        hour,
        total_accidents,
        total_fatalities
    FROM fatalities_hours_days;
    """
)

In [57]:
fatalities_by_hours.to_dataframe()

,day_week,hour,total_accidents,total_fatalities
0,Monday,0,182,204
1,Monday,1,149,161
2,Monday,2,157,165
3,Monday,3,100,113
4,Monday,4,115,123
...,...,...,...,...
163,Sunday,19,356,389
164,Sunday,20,336,364
165,Sunday,21,331,373
166,Sunday,22,293,315


### 6. Cars Involved in the accidents

#### What types of vehicles are most commonly involved in fatal traffic accidents?

In [63]:
fatalities_by_vehicles = client.query(
    """
    SELECT
        make,
        COUNT(*) AS total_accidents,
        SUM(fatals) AS total_fatalities
    FROM NHTSA.accidents AS a
    INNER JOIN NHTSA.vehicles AS v
        ON a.st_case = v.st_case
    GROUP BY make
    ORDER BY total_accidents DESC;
    """
)

In [64]:
fatalities_by_vehicles.to_dataframe()

,make,total_accidents,total_fatalities
0,Chevrolet,7183,8011
1,Toyota,5352,5959
2,None,4960,5215
3,Honda,4542,4975
4,Nissan/Datsun,3346,3788
...,...,...,...
62,Moto-Guzzi,2,2
63,Thomas Built,1,1
64,American Motors,1,1
65,Daihatsu,1,1
